<a href="https://colab.research.google.com/github/peares/CRE-NS3/blob/master/RL1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2


In [27]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [21]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
    
    
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = 38 + random.randint(-3,3)
        # Reset shower time
        self.shower_length = 60 
        return self.state

In [22]:
actions

2

In [26]:
episode = 10
for episode in range(1,episode+1):
  states = env.reset()
  done = False
  score = 0

  while not done:
    env.render()
    action = random.choices([0,1])
    n_state, reward, done, info = env.step(action)
    Score+=reward
  print('Episode:{} Score:{}'.format(episode, score))


NoSuchDisplayException: ignored